In [43]:
import requests
from requests.auth import HTTPBasicAuth
from datetime import datetime

auth_data = HTTPBasicAuth('YOUR API KEY', 'YOUR API SECRET')

date_format = '%Y-%m-%dT%H:%M:%S.%f'

def get_metrics(organization_id):
    api_url = f'https://console.cratedb.cloud/api/v2/organizations/{organization_id}/metrics/prometheus/'
    headers = {'accept': 'plain/text'}
    response = requests.get(api_url, auth=auth_data)
    return response.text

def num_shards(metrics, cluster_id):
    api_url = f"https://console.cratedb.cloud/api/v2/organizations/{organization_id}/metrics/prometheus/"
    headers = {'accept': 'plain/text'}
    response = requests.get(api_url, auth=auth_data, headers=headers)
    if response.status_code == 200:
        lines = response.text.split('\n')
        total_shard_count = 0
        shard_count_instances = 0
        for line in lines:
            if "crate_node" in line and "shard_stats" in line and cluster_id in line and "property=\"total\"" in line:
                shard_count = float(line.split()[1])
                total_shard_count += shard_count
                shard_count_instances += 1
        if shard_count_instances == 0:
            return None  # Return None if no shard counts were found
        else:
            return total_shard_count / shard_count_instances  # Calculate and return the average
    else:
        print(f"Failed to retrieve metrics. Status code: {response.status_code}")
        return None

def get_cluster_status(cluster_id):
    api_url = f'https://console.cratedb.cloud/api/v2/clusters/{cluster_id}/'
    return requests.get(api_url, auth=auth_data).json()

def get_cluster_running_op(cluster_status, cluster_id):
    if (datetime.now() - datetime.strptime(cluster_status['health']['last_seen'], date_format)).seconds > 30:
        cluster_status = get_cluster_status(cluster_id)
    return cluster_status['health']['running_operation']

def get_cluster_num_nodes(cluster_status, cluster_id):
    if (datetime.now() - datetime.strptime(cluster_status['health']['last_seen'], date_format)).seconds > 30:
        cluster_status = get_cluster_status(cluster_id)
    return cluster_status['num_nodes'] - 1


In [45]:
import time

delay_seconds = 5
max_num_shard = 30

organization_id = 'FILL IN YOUR ORGANIZATION ID'
cluster_id = 'FILL IN YOUR CLUSTER ID'

shard_count = num_shards(metrics, cluster_id)
print("Current shard count:", shard_count)

api_url = f'https://console.cratedb.cloud/api/v2/clusters/{cluster_id}/scale/'

num_nodes = get_cluster_num_nodes(cluster_status, cluster_id)
print(num_nodes)
num = num_shards(metrics, cluster_id)
print (num)

while True:
    metrics = get_metrics(organization_id)  # Pass organization_id to get_metrics function
    cluster_status = get_cluster_status(cluster_id)
    num = num_shards(metrics, cluster_id)
    if num is not None:  # Check if num is not None
        print(f'Current avg number of shards: {num}')
        if num > (0.8 * max_num_shard):
            num_nodes = get_cluster_num_nodes(cluster_status, cluster_id)  # Pass cluster_id to get_cluster_num_nodes function
            print(f'Start scaling out from {num_nodes + 1} to {num_nodes + 2}')
            requests.put(api_url, json={'product_unit':num_nodes + 1}, auth=auth_data)
            time.sleep(delay_seconds)
            while get_cluster_running_op(cluster_status, cluster_id) != '':
                cluster_status = get_cluster_status(cluster_id)
                continue
            print('Scaled up successfully!')
                
        elif num < (0.5 * max_num_shard):
            num_nodes = get_cluster_num_nodes(cluster_status, cluster_id)  # Pass cluster_id to get_cluster_num_nodes function
            print(f'Start scaling down from {num_nodes + 1} to {num_nodes}')
            requests.put(api_url, json={'product_unit':num_nodes - 1}, auth=auth_data)
            time.sleep(delay_seconds)
            while get_cluster_running_op(cluster_status, cluster_id) != '':
                cluster_status = get_cluster_status(cluster_id)
                continue
            print('Scaled down successfully!')
        else:
            print('Nothing to do!')
    else:
        print('Failed to retrieve shard metrics.')
    time.sleep(delay_seconds)



Current shard count: 16.8
4
16.8
Current avg number of shards: 16.8
Nothing to do!
Current avg number of shards: 16.8
Nothing to do!
Current avg number of shards: 16.8
Nothing to do!
Current avg number of shards: 16.8
Nothing to do!
Current avg number of shards: 16.8
Nothing to do!
Current avg number of shards: 16.8
Nothing to do!
Current avg number of shards: 16.8
Nothing to do!
Current avg number of shards: 16.8
Nothing to do!
Current avg number of shards: 16.8
Nothing to do!
Current avg number of shards: 16.8
Nothing to do!
Current avg number of shards: 16.8
Nothing to do!
Current avg number of shards: 16.8
Nothing to do!
Current avg number of shards: 16.8
Nothing to do!
Current avg number of shards: 16.8
Nothing to do!
Current avg number of shards: 16.8
Nothing to do!
Current avg number of shards: 16.8
Nothing to do!
Current avg number of shards: 16.8
Nothing to do!
Current avg number of shards: 16.8
Nothing to do!
Current avg number of shards: 16.8
Nothing to do!
Current avg numbe

KeyboardInterrupt: 